### Creating the Annotation file based on the overdose rate(Considering entire US)

In [7]:
# Import libraries
import os
import geopandas as gpd
import pandas as pd

In [8]:
overdose_df = pd.read_excel('/home/h6x/git_projects/ornl-svi-data-processing/raw_data/HepVu_County_Opioid_Indicators_05DEC22.xlsx',dtype={'GEO ID': str} )

In [9]:
overdose_df.head(3)

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,1001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,1003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,1005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6


In [10]:
overdose_df[overdose_df['GEO ID']=='36081']

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
1869,36081,NY,Queens County,12.7,6.7,7.2,11.4,12.3,11.6,11.7,18.1


In [11]:
overdose_df = overdose_df[['GEO ID','County Name','Narcotic Overdose Mortality Rate 2018']]

In [12]:
overdose_df = overdose_df.rename(columns={'Narcotic Overdose Mortality Rate 2018': 'NOD_Rate','GEO ID':'STCNTY'})

In [13]:
overdose_df.shape

(3221, 3)

In [14]:
# Calculate the min and max of the column
min_value = overdose_df['NOD_Rate'].min()
max_value = overdose_df['NOD_Rate'].max()

In [15]:
min_value

-9.0

In [16]:
max_value

114.4

In [17]:
# Assuming negative overdose rate means this data is missing
len(overdose_df[overdose_df['NOD_Rate'] < 0])

89

In [18]:
# lets not consider the missing data
overdose_df = overdose_df[overdose_df['NOD_Rate'] >= 0]

In [19]:
overdose_df.shape

(3132, 3)

In [20]:
# overdose_df['OD_class'] = pd.qcut(overdose_df['NOD_Rate'], q=[0,0.90, 1], labels=['0', '1'])
overdose_df['OD_class'] = pd.qcut(overdose_df['NOD_Rate'], q=[0,0.20,0.40,0.60,0.80, 1], labels=['0','1','2','3','4'])

In [21]:
overdose_df = overdose_df.reset_index(drop=True)

In [22]:
overdose_df['STCNTY'] = overdose_df['STCNTY'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [23]:
overdose_df.sort_values(by=['NOD_Rate'], ascending=False).head(5)

,STCNTY,County Name,NOD_Rate,OD_class
2987,54011,Cabell County,114.4,4
1206,24510,Baltimore city,108.8,4
2984,54005,Boone County,94.7,4
3005,54047,McDowell County,85.1,4
1806,35039,Rio Arriba Coun,80.0,4


In [25]:
overdose_df.to_csv('/home/h6x/git_projects/ornl-svi-data-processing/processed_data/annotations/annotations_2018_5_classes.csv', index=False)